In [1]:
!pip install chart_studio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.4/64.4 KB 4.2 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import numpy as np 
import pandas as pd 
import pickle
from sklearn.manifold import TSNE
# from umap import UMAP

import matplotlib.pyplot as plt 
import seaborn as sns 
import plotly.express as px

In [20]:
import chart_studio
import chart_studio.plotly as py 
import chart_studio.tools as tls



In [19]:
plotly_user = 'nipdep'
plotly_key = 'BNQUTzoP5YBLtc8uggat'
chart_studio.tools.set_credentials_file(username=plotly_user, api_key=plotly_key)

In [4]:
def read_I3D_pkl(loc,feat_size="400"):
  if feat_size == "400":
    feat_index = 1
  elif feat_size == "2048":
    feat_index = 0
  else:
    raise NotImplementedError()

  with open(loc,"rb") as f0:
    __data = pickle.load(f0)

  label = []
  prototype = []
  for k,v in __data.items():
    label.append(k)
    all_arr = [x[feat_index] for x in v]
    all_arr = np.asarray(all_arr).mean(axis=0)
    prototype.append(all_arr)

  label = np.asarray(label)
  prototype = np.array(prototype)
  return {"activity":label, "features":prototype}

In [5]:
data_root = '/content/drive/MyDrive/22_FYP42 - Zero-shot Explainable HAR/Datasets/Consolidated/PAMPA2'

In [7]:
I3D_data_path  = data_root + '/I3D/video_feat.pkl'
video_data = read_I3D_pkl(I3D_data_path, feat_size="400")
video_classes, attr_mat = video_data['activity'], video_data['features']

In [8]:
with open(I3D_data_path, "rb") as f0:
    data = pickle.load(f0)

In [9]:
data.keys()

dict_keys(['Nordic walking', 'running', 'ironing', 'ascending stairs', 'vacuum cleaning', 'walking', 'watching TV', 'computer work', 'playing soccer', 'car driving', 'house cleaning', 'standing', 'folding laundry', 'rope jumping', 'lying', 'sitting', 'descending stairs', 'cycling'])

In [10]:
emb_400 = np.array([i[0] for i in data['running']])
emb_400.shape

(15, 2048)

In [11]:
video_classes

array(['Nordic walking', 'running', 'ironing', 'ascending stairs',
       'vacuum cleaning', 'walking', 'watching TV', 'computer work',
       'playing soccer', 'car driving', 'house cleaning', 'standing',
       'folding laundry', 'rope jumping', 'lying', 'sitting',
       'descending stairs', 'cycling'], dtype='<U17')

In [12]:
all_embs = []
all_ys = []
for k, v in data.items():
    emb_400 = np.array([i[0] for i in v])
    yn = emb_400.shape[0]
    all_ys.extend([k,]*yn)
    all_embs.extend(emb_400)


In [13]:
all_embs_np = np.array(all_embs)
all_ys_np = np.array(all_ys)

all_embs_np.shape, all_ys_np.shape

((224, 2048), (224,))

In [16]:
def plot_tsne(X, y):
    tnse = TSNE(n_components=2, init='random', learning_rate='auto', perplexity=15, n_iter=1000)
    cmp_data = tnse.fit_transform(X)

    cmp_df = pd.DataFrame(data=cmp_data, columns=['x', 'y'])
    cmp_df['action'] = y

    fig = px.scatter(cmp_df, x='x', y='y', color='action', width=800, height=800, title='t-SNE')
    fig.show()
    return fig

In [17]:
tsne_fig = plot_tsne(all_embs_np, all_ys_np)

In [21]:
py.plot(tsne_fig, filename='TSNE_I3D_plot', auto_open=False)

PlotlyRequestError: ignored